<a href="https://colab.research.google.com/github/ErykCud/MLinFin2024/blob/main/CASE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner
!pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=2010e82d98b0d02dfc2e54b99ac518aedb93e00834d913b9bf83f6318fe1fc67
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [3]:
import datetime as dt
import yfinance as yf
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


SIMPLE RNN, TEST. -9% procent zwrotu. (słabo)

In [4]:

#ustawianie danych treningowych, biorę dane z zeszłego roku
train_start_date = dt.datetime(2022, 1, 1)
train_end_date = dt.datetime(2023, 12, 31)

test_start_date = dt.datetime(2024, 1, 1)
test_end_date = dt.datetime(2024, 6, 5)

train_data = yf.download("AAPL", start=train_start_date, end=train_end_date)
test_data = yf.download("AAPL", start=test_start_date, end=test_end_date)

train_data = train_data.iloc[:, :1]
test_data = test_data.iloc[:, :1]


dataset_train = train_data.Open.values
dataset_test = test_data.Open.values


dataset_train = np.reshape(dataset_train, (-1, 1))
dataset_test = np.reshape(dataset_test, (-1, 1))

#SKALER
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.transform(dataset_test)


X_train = np.array([scaled_train[i-1:i] for i in range(1, len(scaled_train))])
y_train = scaled_train[1:]


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))


regressor = Sequential()

#SIEĆ RNN
regressor.add(SimpleRNN(units=50, activation="tanh", return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units=50, activation="tanh", return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units=50, activation="tanh", return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units=50))
regressor.add(Dropout(0.2))


regressor.add(Dense(units=1, activation='sigmoid'))

regressor.compile(optimizer='adam', loss="mean_squared_error")


regressor.fit(X_train, y_train, epochs=20, batch_size=2)

# Backtesting
class RNNForecastStrategy(Strategy):
    def init(self):
        self.signal = self.I(self.predict_prices, self.data.Open)

    def predict_prices(self, data):

        data = np.array(data).reshape(-1, 1)


        scaled_data = scaler.transform(data)
        X = np.array([scaled_data[i-1:i] for i in range(1, len(scaled_data))])
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))


        predicted = regressor.predict(X)
        predicted = scaler.inverse_transform(predicted)


        predicted_padded = np.zeros(len(data))
        predicted_padded[1:] = predicted.flatten()
        predicted_padded[0] = data[0]

        return predicted_padded

    def next(self):
        if crossover(self.signal, self.data.Close):
            self.buy()
        elif crossover(self.data.Close, self.signal):
            self.sell()
#ODPALENIE BAKTESTU
data_for_backtest = yf.download("AAPL", start=test_start_date, end=test_end_date)
bt = Backtest(data_for_backtest, RNNForecastStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Epoch 1/20
250/250 [==============================] - 5s 5ms/step - loss: 0.0188
Epoch 2/20
250/250 [==============================] - 1s 5ms/step - loss: 0.0056
Epoch 3/20
250/250 [==============================] - 1s 4ms/step - loss: 0.0048
Epoch 4/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0045
Epoch 5/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0042
Epoch 6/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0046
Epoch 7/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0038
Epoch 8/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0038
Epoch 9/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0042
Epoch 10/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0041
Epoch 11/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0037
Epoch 12/20
250/250 [==============================] - 1s 3ms/step - loss: 0.0041
Epoch 13/20
250/250 [====

[*********************100%%**********************]  1 of 1 completed


4/4 [==============================] - 0s 3ms/step


<ipython-input-4-d80eaa331bd5>:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_padded[0] = data[0]
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/

GridPlot(id='p1315', ...)

LSTM Z HYPERPARAMETRYZACJĄ -7% ZWROTU, WCIĄŻ SŁABO.

In [5]:


#DANE CZASOWE
train_start_date = dt.datetime(2023, 1, 1)
train_end_date = dt.datetime(2023, 12, 31)
test_start_date = dt.datetime(2024, 1, 1)
test_end_date = dt.datetime(2024, 6, 5)

train_data = yf.download("AAPL", start=train_start_date, end=train_end_date)
test_data = yf.download("AAPL", start=test_start_date, end=test_end_date)

#
dataset_train = train_data['Open'].values.reshape(-1, 1)
dataset_test = test_data['Open'].values.reshape(-1, 1)

# NORMALIZACJA
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.transform(dataset_test)

def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

seq_length = 20
X_train, y_train = create_sequences(scaled_train, seq_length)
X_test, y_test = create_sequences(scaled_test, seq_length)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


def build_model(hp):
    input_shape = (seq_length, 1)
    inputs = Input(shape=input_shape)

    d_model = hp.Int('d_model', min_value=32, max_value=128, step=32)
    num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2)
    ff_units = hp.Int('ff_units', min_value=32, max_value=128, step=32)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)

    attention = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
    attention = Dropout(dropout_rate)(attention)
    attention = LayerNormalization(epsilon=1e-6)(attention + inputs)

    outputs = Dense(ff_units, activation='relu')(attention)
    outputs = Dropout(dropout_rate)(outputs)
    outputs = LayerNormalization(epsilon=1e-6)(outputs + attention)

    outputs = Dense(1)(outputs[:, -1, :])

    model = Model(inputs=inputs, outputs=outputs)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    model.compile(optimizer=Adam(learning_rate), loss='mean_squared_error')

    return model
#HYPERPARAMETRYZACJA
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='transformer_hyperparam_tuning',
    project_name='AAPL_Transformer'
)

# VALUIDACJA
X_train_partial = X_train[:int(0.8 * len(X_train))]
y_train_partial = y_train[:int(0.8 * len(y_train))]
X_val = X_train[int(0.8 * len(X_train)):]
y_val = y_train[int(0.8 * len(y_train)):]

tuner.search(X_train_partial, y_train_partial, epochs=20, validation_data=(X_val, y_val), batch_size=32)

# WEŹ HYPERPARAMETRY
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the transformer layers is {best_hps.get('d_model')},
the number of heads is {best_hps.get('num_heads')},
the feed-forward units is {best_hps.get('ff_units')},
the dropout rate is {best_hps.get('dropout_rate')},
and the learning rate is {best_hps.get('learning_rate')}.
""")


model = tuner.hypermodel.build(best_hps)

# WYTRENUJ MODEL
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# BAKTESTING
class TransformerForecastStrategy(Strategy):
    def init(self):
        self.signal = self.I(self.predict_prices, self.data.Open)

    def predict_prices(self, data):

        data = np.array(data).reshape(-1, 1)


        scaled_data = scaler.transform(data)
        X = np.array([scaled_data[i-seq_length:i] for i in range(seq_length, len(scaled_data))])


        if X.size > 0:
            predicted = model.predict(X)
            predicted = scaler.inverse_transform(predicted)

            predicted_padded = np.zeros(len(data))
            predicted_padded[seq_length:] = predicted.flatten()
            predicted_padded[:seq_length] = data[:seq_length].flatten()

            return predicted_padded
        else:
            return np.zeros(len(data))

    def next(self):
        if crossover(self.signal, self.data.Close):
            self.buy()
        elif crossover(self.data.Close, self.signal):
            self.sell()

# TEN BAKTEST
data_for_backtest = yf.download("AAPL", start=test_start_date, end=test_end_date)
bt = Backtest(data_for_backtest, TransformerForecastStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()
print(stats)


Trial 10 Complete [00h 00m 13s]
val_loss: 0.014904763549566269

Best val_loss So Far: 0.014904763549566269
Total elapsed time: 00h 02m 11s

The optimal number of units in the transformer layers is 32,
the number of heads is 6,
the feed-forward units is 64,
the dropout rate is 0.5,
and the learning rate is 0.001.

Epoch 1/20
8/8 [==============================] - 2s 48ms/step - loss: 0.9748 - val_loss: 0.6682
Epoch 2/20
8/8 [==============================] - 0s 17ms/step - loss: 0.2423 - val_loss: 0.0262
Epoch 3/20
8/8 [==============================] - 0s 15ms/step - loss: 0.0933 - val_loss: 0.0139
Epoch 4/20
8/8 [==============================] - 0s 16ms/step - loss: 0.0982 - val_loss: 0.0298
Epoch 5/20
8/8 [==============================] - 0s 17ms/step - loss: 0.0691 - val_loss: 0.0657
Epoch 6/20
8/8 [==============================] - 0s 16ms/step - loss: 0.0757 - val_loss: 0.0671
Epoch 7/20
8/8 [==============================] - 0s 19ms/step - loss: 0.0529 - val_loss: 0.0458
Epoch 

[*********************100%%**********************]  1 of 1 completed


3/3 [==============================] - 0s 13ms/step


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 00:00:00
End                       2024-06-04 00:00:00
Duration                    154 days 00:00:00
Exposure Time [%]                   98.130841
Equity Final [$]                  9293.013199
Equity Peak [$]                  10923.213165
Return [%]                          -7.069868
Buy & Hold Return [%]                 4.69188
Return (Ann.) [%]                  -15.859668
Volatility (Ann.) [%]               17.656604
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -14.92418
Avg. Drawdown [%]                   -3.816834
Max. Drawdown Duration       52 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -7.071132
Worst Trade [%]                     -7.071132
Avg. Trade [%]                    

In [ ]:
# Running backtest
data_for_backtest = yf.download("AAPL", start=test_start_date, end=test_end_date)
bt = Backtest(data_for_backtest, RNNForecastStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()

[*********************100%%**********************]  1 of 1 completed


2/2 [==============================] - 0s 125ms/step


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1660', ...)

In [ ]:
import datetime as dt
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.optimizers import Adam
from keras_tuner import RandomSearch, HyperParameters
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
#WYBIERAM DANE
train_start_date = dt.datetime(2023, 1, 1)
train_end_date = dt.datetime(2023, 12, 31)
test_start_date = dt.datetime(2024, 1, 1)
test_end_date = dt.datetime(2024, 6, 5)

train_data = yf.download("AAPL", start=train_start_date, end=train_end_date)
test_data = yf.download("AAPL", start=test_start_date, end=test_end_date)

dataset_train = train_data['Open'].values.reshape(-1, 1)
dataset_test = test_data['Open'].values.reshape(-1, 1)

# NNORMALIZACJA
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.transform(dataset_test)

def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

seq_length = 60
X_train, y_train = create_sequences(scaled_train, seq_length)
X_test, y_test = create_sequences(scaled_test, seq_length)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the model building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=50, max_value=200, step=50),
                   return_sequences=True,
                   input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(LSTM(units=hp.Int(f'units_{i}', min_value=50, max_value=200, step=50), return_sequences=True))
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(LSTM(units=hp.Int('units_last', min_value=50, max_value=200, step=50)))
    model.add(Dropout(hp.Float('dropout_last', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(units=1))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error')
    return model


tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='lstm_hyperparam_tuning',
    project_name='AAPL_LSTM'
)


X_train_partial = X_train[:int(0.8 * len(X_train))]
y_train_partial = y_train[:int(0.8 * len(y_train))]
X_val = X_train[int(0.8 * len(X_train)):]
y_val = y_train[int(0.8 * len(y_train)):]

#HYPERPARAMETRYZACJA
tuner.search(X_train_partial, y_train_partial, epochs=10, validation_data=(X_val, y_val), batch_size=32)


best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first LSTM layer is {best_hps.get('units')},
the optimal dropout rate is {best_hps.get('dropout')},
the optimal number of layers is {best_hps.get('num_layers')},
and the optimal learning rate is {best_hps.get('learning_rate')}.
""")

model = tuner.hypermodel.build(best_hps)


model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

#BAKTESTING
class LSTMForecastStrategy(Strategy):
    def init(self):
        self.signal = self.I(self.predict_prices, self.data.Open)

    def predict_prices(self, data):

        data = np.array(data).reshape(-1, 1)

        scaled_data = scaler.transform(data)
        X = np.array([scaled_data[i-seq_length:i] for i in range(seq_length, len(scaled_data))])
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))


        if X.size > 0:
            predicted = model.predict(X)
            predicted = scaler.inverse_transform(predicted)

            predicted_padded = np.zeros(len(data))
            predicted_padded[seq_length:] = predicted.flatten()
            predicted_padded[:seq_length] = data[:seq_length].flatten()


            return predicted_padded
        else:
            return np.zeros(len(data))

    def next(self):
        if crossover(self.signal, self.data.Close):
            self.buy()
        elif crossover(self.data.Close, self.signal):
            self.sell()

# Run the backtest
data_for_backtest = yf.download("AAPL", start=test_start_date, end=test_end_date)
bt = Backtest(data_for_backtest, LSTMForecastStrategy, cash=10000, commission=.002)
stats = bt.run()
bt.plot()
print(stats)


Trial 10 Complete [00h 00m 57s]
val_loss: 0.0033792165340855718

Best val_loss So Far: 0.0033792165340855718
Total elapsed time: 00h 11m 15s

The optimal number of units in the first LSTM layer is 150,
the optimal dropout rate is 0.2,
the optimal number of layers is 1,
and the optimal learning rate is 0.0001.

Epoch 1/50
6/6 [==============================] - 10s 542ms/step - loss: 0.5278 - val_loss: 0.3734
Epoch 2/50
6/6 [==============================] - 2s 283ms/step - loss: 0.3121 - val_loss: 0.1858
Epoch 3/50
6/6 [==============================] - 2s 288ms/step - loss: 0.1173 - val_loss: 0.0353
Epoch 4/50
6/6 [==============================] - 2s 282ms/step - loss: 0.0215 - val_loss: 0.0330
Epoch 5/50
6/6 [==============================] - 3s 496ms/step - loss: 0.0442 - val_loss: 0.0188
Epoch 6/50
6/6 [==============================] - 3s 409ms/step - loss: 0.0175 - val_loss: 0.0178
Epoch 7/50
6/6 [==============================] - 2s 280ms/step - loss: 0.0213 - val_loss: 0.0216
E

[*********************100%%**********************]  1 of 1 completed


2/2 [==============================] - 1s 54ms/step


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 00:00:00
End                       2024-06-04 00:00:00
Duration                    154 days 00:00:00
Exposure Time [%]                   98.130841
Equity Final [$]                  9177.363816
Equity Peak [$]                  10985.403778
Return [%]                          -8.226362
Buy & Hold Return [%]                 4.69188
Return (Ann.) [%]                  -18.304988
Volatility (Ann.) [%]               18.511314
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.458566
Avg. Drawdown [%]                   -4.108145
Max. Drawdown Duration       52 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                       6.065218
Worst Trade [%]                    -14.340184
Avg. Trade [%]                    

Trial 10 Complete [00h 01m 51s]
val_loss: 0.007506788009777665

Best val_loss So Far: 0.00478925765492022
Total elapsed time: 00h 21m 21s

The optimal number of units in the first LSTM layer is 200,
the optimal dropout rate is 0.30000000000000004,
the optimal number of layers is 2,
and the optimal learning rate is 0.01.

Epoch 1/50
15/15 [==============================] - 13s 398ms/step - loss: 1.8744 - val_loss: 0.0210
Epoch 2/50
15/15 [==============================] - 6s 395ms/step - loss: 0.0860 - val_loss: 0.0856
Epoch 3/50
15/15 [==============================] - 4s 286ms/step - loss: 0.0826 - val_loss: 0.0888
Epoch 4/50
15/15 [==============================] - 5s 319ms/step - loss: 0.0740 - val_loss: 0.0671
Epoch 5/50
15/15 [==============================] - 6s 367ms/step - loss: 0.0811 - val_loss: 0.0824
Epoch 6/50
15/15 [==============================] - 4s 286ms/step - loss: 0.0744 - val_loss: 0.0787
Epoch 7/50
15/15 [==============================] - 5s 347ms/step - loss: 0.

[*********************100%%**********************]  1 of 1 completed


2/2 [==============================] - 2s 53ms/step


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 00:00:00
End                       2024-06-04 00:00:00
Duration                    154 days 00:00:00
Exposure Time [%]                   98.130841
Equity Final [$]                  9064.109642
Equity Peak [$]                  10992.153778
Return [%]                          -9.358904
Buy & Hold Return [%]                 4.69188
Return (Ann.) [%]                  -20.659534
Volatility (Ann.) [%]               18.474644
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -17.540185
Avg. Drawdown [%]                   -4.228325
Max. Drawdown Duration       52 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    8
Win Rate [%]                             25.0
Best Trade [%]                       8.947728
Worst Trade [%]                    -17.949835
Avg. Trade [%]                    